## Data Acquisition: Extract data from the web

Data acquisition is a crucial set for developing an information retrieval system. With the boom of the internet, there is so much data lying on the web in the form of websites. 
There are many ways to extract data from the web, and APIs are probably the best way to extract data from a website. 
Most big websites like Twitter, Facebook, Amazon, and New York Times provide APIs to access their data.
But not all websites have an API. 
Some websites don't provide one because of privacy concerns or they lack technical use-case to provide one. 

**Web scraping** is a technique of extracting information from websites. 
It focuses on transforming partially structures data (HTML format) on the web into structured data (database or spreadsheet).

Python has a rich ecosystem to scrape data from the web, and it is somewhat easy to use once you get used to the API and understand the hierarchical nature of web documents.

The library `BeautifulSoup` assists this task.

#### LIbraries used

**`requests`**: 
This library is used for fetching data from web pages. 
[Click here for documentation](http://docs.python-requests.org/en/master/)

**`BeautifulSoup`**: 
Use this library to extract tables, lists, paragraph from html web pages. 
It also allows filters to extract information from web pages. 
[Click here for documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)


**Note**

For a small and less complex project, Beautiful Soup can do the task pretty well. For a beginner, it's a good option. There are other packages available for handling more complex scraping tasks: 

1. [Selenium](https://selenium-python.readthedocs.io/): When you are dealing with Javascript featured website then Selenium could be the best choice. 
2. [Scrapy](https://scrapy.org/): For a large/complex project, Scrapy is the best choice. It's much easier to create web-crawler with Scrapy. 

---

### Example: https://en.wikipedia.org/wiki/List_of_World_Series_champions

Please click on the URL and review the page, visually.
Then proceed with the code below.


In [1]:
#import the library to query a website
import requests
# import Beautiful soup library to access functions to parse the data returned from the website
from bs4 import BeautifulSoup

In [2]:
# specify the url
url = "https://en.wikipedia.org/wiki/List_of_World_Series_champions"

In [3]:
# Open website URL and return the html to the variable 'response'
response = requests.get(url)

The response we get from web is typically html content. 
We can read the content of the server's response. 
Below, when a `BeautifulSoup` object is created from an html response, we explicitly reference the text format(`response.text`).

The default encoding format is 'UTF-8' as shown below. 

[Click here for additional documentations about the response object.](http://docs.python-requests.org/en/master/user/quickstart/#response-content)

In [4]:
response.encoding

'UTF-8'

In [5]:
response

<Response [200]>

In [6]:
# Parse the html in the 'response' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(response.text, "html")

### Basic Inspection
Use `prettify` function to print the data in its nested html structured format.

In [7]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of World Series champions - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"789440d1-b023-4c17-8424-f64a6f1f6d44","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_World_Series_champions","wgTitle":"List of World Series champions","wgCurRevisionId":1043339945,"wgRevisionId":1043339945,"wgArticleId":7599168,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description matches Wikidata","Articles with h

We need to extract the table which has list of all baseball world series champions. 

This table should be present in one of the html tags.

We can work with the tags to extract data present in them.  
"**soup.tag**": will return the content between opening and closing tag including tag. 

Additionally, the `.string` value is the data between the tags.
Compare the two cells below.

In [8]:
soup.title

<title>List of World Series champions - Wikipedia</title>

In [9]:
# Return string within given tag 
soup.title.string

'List of World Series champions - Wikipedia'

**Identify the html tag**: 
The data is in a table. 
You can use inspect element option when you right click the mouse to identify the tag which has the data. 

 * [Additional guide on webpage inspection](../resources/AnalyzingHTMLwithTheWebInspector.pdf)


<img src="../images/table.png">

**Find the right table:** 
As we are seeking a table to extract information about baseball champions, we should identify the right table first. 
Let’s write the command to extract information within all table tags. 

In [10]:
all_tables=soup.find_all('table')

Now to identify the right table, we will use attribute “class” of table and use it to filter the right table.
In _Chrome_, you can check the class name by **right clicking** on the required table of web page, then  `–> Inspect element –> Copy` the class name OR go through the output of above command find the class name of right table.

In [11]:
right_table=soup.find('table', class_='wikitable sortable plainrowheaders')
right_table

<table class="wikitable sortable plainrowheaders">
<tbody><tr>
<th scope="col">Year
</th>
<th scope="col">Winning team
</th>
<th scope="col">Manager
</th>
<th class="unsortable" scope="col">Games
</th>
<th scope="col">Losing team
</th>
<th scope="col">Manager
</th></tr>
<tr>
<th scope="row" style="text-align:center"><a href="/wiki/1903_World_Series" title="1903 World Series">1903</a>
</th>
<td align="left" style="background:#fcc;"><a href="/wiki/1903_Boston_Americans_season" title="1903 Boston Americans season">Boston Americans</a><small> (1, 1–0)</small></td>
<td align="left"><span data-sort-value="Collins, Jimmy"><span class="vcard"><span class="fn"><a href="/wiki/Jimmy_Collins" title="Jimmy Collins">Jimmy Collins</a></span></span></span></td>
<td>5–3<sup class="reference" id="ref_BestOf9V"><a href="#endnote_BestOf9V">[V]</a></sup></td>
<td align="left" style="background:#d0e7ff;"><a href="/wiki/1903_Pittsburgh_Pirates_season" title="1903 Pittsburgh Pirates season">Pittsburgh Pirates

When we are ready to process the data (scrape it), we will accumulate the columns, then assemble the final columns into a dataframe.

In [12]:
#Generate lists
Year=[]
Winning_team=[]
Winning_Manager=[]
Games=[]
Losing_team=[]
Losing_Manager=[]
Ref=[]

# skip first iteration as we dont need headers 
for row in right_table.findAll("tr")[1:]: 
#     print(row)
    
    game_year=row.findAll('th') # To store game year which is in <th> tag
    cells = row.findAll('td') # To store all other details
    
    if len(cells)>2: # Only extract information if there is table body not heading
        Year.append(game_year[0].find(text=True))
        Winning_team.append(cells[0].find(text=True))
        Winning_Manager.append(cells[1].find(text=True))
        Games.append(cells[2].find(text=True))
        Losing_team.append(cells[3].find(text=True))
        Losing_Manager.append(cells[4].find(text=True))

Now we can push the extracted data into a DataFrame.

Here, we need to iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list. 
Let’s first look at the HTML structure of the table

In [13]:
#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(Year,columns=['Year'])
df['Winning_team']=Winning_team
df['Winning_Manager']=Winning_Manager
df['Games']=Games
df['Losing_team']=Losing_team
df['Losing_Manager']=Losing_Manager

df.head()

,Year,Winning_team,Winning_Manager,Games,Losing_team,Losing_Manager
0,1903,Boston Americans,Jimmy Collins,5–3,Pittsburgh Pirates,Fred Clarke
1,1905,New York Giants,John McGraw,4–1,Philadelphia Athletics,Connie Mack
2,1906,Chicago White Sox,Fielder Jones,4–2,Chicago Cubs,Frank Chance
3,1907,Chicago Cubs,Frank Chance,4–0–(1),Detroit Tigers,Hugh Jennings
4,1908,Chicago Cubs,Frank Chance,4–1,Detroit Tigers,Hugh Jennings



We can see that we have a Pandas dataframe filled with the data.
We can now proceed to do data cleaning, manipulation, exploration, and persistence.



# Save your notebook, then `File > Close and Halt`